In [30]:
import warnings

import numpy as np
from imblearn.pipeline import Pipeline

warnings.simplefilter(action='ignore')

import pandas as pd
import seaborn as sns

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix

sns.set(rc={'figure.figsize':(10, 8)})

In [31]:
df_ForestImputer = pd.read_csv('../../Train_music_ForestImputer.csv')
df_ForestImputer

id  target  device_type  manufacturer_category  os_category  \
0      4.810120e+10     0.0          7.0                  592.0          3.0   
1      4.810425e+10     0.0          7.0                 1057.0          3.0   
2      4.811813e+09     0.0          7.0                 1057.0          3.0   
3      4.810105e+10     0.0          7.0                  745.0          3.0   
4      4.830019e+09     0.0          7.0                 1057.0          3.0   
...             ...     ...          ...                    ...          ...   
19995  4.894477e+09     1.0          7.0                 1057.0          3.0   
19996  4.875985e+09     0.0          7.0                  592.0          3.0   
19997  4.811248e+09     0.0          7.0                  728.0          3.0   
19998  4.896164e+09     0.0          7.0                 1057.0          3.0   
19999  4.852292e+09     0.0          7.0                 1228.0          3.0   

       sim_count  tp_flag        lt  block_flag  days_exp  ...  \
0            2.0      0.0  0.632435         0.0     108.0  ...   
1            2.0      0.0  0.540345         0.0     330.0  ...   
2            2.0      0.0  0.951582         0.0     344.0  ...   
3            2.0      0.0  0.659047         0.0     361.0  ...   
4            2.0      0.0  0.926471         0.0     199.0  ...   
...          ...      ...       ...         ...       ...  ...   
19995        2.0      0.0  0.707827         0.0     221.0  ...   
19996        2.0      0.0  0.824135         0.0     355.0  ...   
19997        1.0      0.0  0.924764         0.0     208.0  ...   
19998        2.0      0.0  0.716732         0.0     324.0  ...   
19999        1.0      0.0  0.880862         0.0     343.0  ...   

       service_5_count_m3  service_6_count_m3  service_7_cost_m3  \
0                     0.0                 0.0                0.0   
1                     0.0                 0.0                0.0   
2                     0.0                 0.0                0.0   
3                     0.0                 0.0                0.0   
4                     0.0                 0.0                0.0   
...                   ...                 ...                ...   
19995                 0.0                 0.0                0.0   
19996                 0.0                 0.0                0.0   
19997                 0.0                 0.0                0.0   
19998                 0.0                 0.0                0.0   
19999                92.0                 0.0                0.0   

       service_7_flag_m3  service_8_count_m3  income_brnd_cont_m3  \
0                    0.0                 0.0            19.971334   
1                    0.0                 0.0            10.894140   
2                    0.0                 0.0             9.894929   
3                    0.0                 0.0             8.183337   
4                    0.0                 0.0             2.217369   
...                  ...                 ...                  ...   
19995                0.0                 0.0            11.649066   
19996                0.0                 0.0             2.159549   
19997                0.0                 0.0            10.999666   
19998                0.0                 0.0            14.756591   
19999                0.0                 0.0             2.745100   

       data_type_1_m3  data_type_2_m3  data_type_3_m3  service_9_flag_m3  
0          127.895751      716.420311      114.566613                1.0  
1          234.292876       80.144736       14.630618                1.0  
2            0.018959        0.000977       57.100249                1.0  
3          189.579215      118.776836       53.537855                1.0  
4          534.214973     1989.913944       29.685412                1.0  
...               ...             ...             ...                ...  
19995      522.172754     2286.497442      452.738112                1.0  
19996      857.21

In [32]:
X = df_ForestImputer.drop(['target'], axis=1)
y = df_ForestImputer.target

In [33]:
X.drop(['id'], axis=1, inplace=True)
X

device_type  manufacturer_category  os_category  sim_count  tp_flag  \
0              7.0                  592.0          3.0        2.0      0.0   
1              7.0                 1057.0          3.0        2.0      0.0   
2              7.0                 1057.0          3.0        2.0      0.0   
3              7.0                  745.0          3.0        2.0      0.0   
4              7.0                 1057.0          3.0        2.0      0.0   
...            ...                    ...          ...        ...      ...   
19995          7.0                 1057.0          3.0        2.0      0.0   
19996          7.0                  592.0          3.0        2.0      0.0   
19997          7.0                  728.0          3.0        1.0      0.0   
19998          7.0                 1057.0          3.0        2.0      0.0   
19999          7.0                 1228.0          3.0        1.0      0.0   

             lt  block_flag  days_exp  service_1_flag  service_1_count  ...  \
0      0.632435         0.0     108.0             0.0              0.0  ...   
1      0.540345         0.0     330.0             0.0              0.0  ...   
2      0.951582         0.0     344.0             0.0              0.0  ...   
3      0.659047         0.0     361.0             0.0              0.0  ...   
4      0.926471         0.0     199.0             1.0             25.0  ...   
...         ...         ...       ...             ...              ...  ...   
19995  0.707827         0.0     221.0             0.0              0.0  ...   
19996  0.824135         0.0     355.0             0.0              0.0  ...   
19997  0.924764         0.0     208.0             0.0              0.0  ...   
19998  0.716732         0.0     324.0             0.0              0.0  ...   
19999  0.880862         0.0     343.0             1.0           1236.0  ...   

       service_5_count_m3  service_6_count_m3  service_7_cost_m3  \
0                     0.0                 0.0                0.0   
1                     0.0                 0.0                0.0   
2                     0.0                 0.0                0.0   
3                     0.0                 0.0                0.0   
4                     0.0                 0.0                0.0   
...                   ...                 ...                ...   
19995                 0.0                 0.0                0.0   
19996                 0.0                 0.0                0.0   
19997                 0.0                 0.0                0.0   
19998                 0.0                 0.0                0.0   
19999                92.0                 0.0                0.0   

       service_7_flag_m3  service_8_count_m3  income_brnd_cont_m3  \
0                    0.0                 0.0            19.971334   
1                    0.0                 0.0            10.894140   
2                    0.0                 0.0             9.894929   
3                    0.0                 0.0             8.183337   
4                    0.0                 0.0             2.217369   
...                  ...                 ...                  ...   
19995                0.0                 0.0            11.649066   
19996                0.0                 0.0             2.159549   
19997                0.0                 0.0            10.999666   
19998                0.0                 0.0            14.756591   
19999                0.0                 0.0             2.745100   

       data_type_1_m3  data_type_2_m3  data_type_3_m3  service_9_flag_m3  
0          127.895751      716.420311      114.566613                1.0  
1          234.292876       80.144736       14.630618                1.0  
2            0.018959        0.000977       57.100249                1.0  
3          189.579215      118.776836       53.537855                1.0  
4          534.214973     1989.913944       29.685412                1.0  
...               ...    

In [34]:
selector_50 = SelectKBest(f_classif, k=50)
selector_50.fit(X, y)
# Get columns to keep and create new dataframe with those only
cols_50 = selector_50.get_support(indices=True)
df_new_50 = X.iloc[:,cols_50]
df_new_50

os_category        lt  is_obl_center  count_sms_source_5     vol_app_5  \
0              3.0  0.632435            0.0                 1.0  3.710730e+05   
1              3.0  0.540345            1.0                 0.0  4.475127e+08   
2              3.0  0.951582            0.0                 3.0  2.293159e+06   
3              3.0  0.659047            0.0                 0.0  8.941500e+04   
4              3.0  0.926471            0.0                 1.0  2.869470e+05   
...            ...       ...            ...                 ...           ...   
19995          3.0  0.707827            0.0                 0.0  8.465320e+05   
19996          3.0  0.824135            1.0                 2.0  2.059290e+08   
19997          3.0  0.924764            1.0                 0.0  1.526081e+08   
19998          3.0  0.716732            0.0                 1.0  6.835147e+06   
19999          3.0  0.880862            1.0                 0.0  1.096679e+09   

       rr_act_type_3  rr_act_type_8  rr_gift_type_3  count_act_type_1  \
0                0.0       0.000000             0.0          1.000000   
1                0.0       0.500000             0.0          4.000000   
2                2.0       1.833333             2.0          5.000000   
3                0.0       0.000000             0.0          1.000000   
4                0.0       0.000000             0.0          6.000000   
...              ...            ...             ...               ...   
19995            0.0       0.000000             0.0          5.000000   
19996            0.0       0.000000             0.0          1.000000   
19997            0.0       0.000000             0.0          7.000000   
19998            0.0       0.000000             0.0          1.057143   
19999            0.0       0.000000             0.0          6.000000   

       count_act_type_3  ...  act_days_count_m3  paym_count_m3  \
0              1.000000  ...               28.0            0.0   
1              1.028571  ...               14.0            1.0   
2              4.000000  ...               24.0            0.0   
3              1.000000  ...                6.0            0.0   
4              1.000000  ...               28.0            0.0   
...                 ...  ...                ...            ...   
19995          1.000000  ...                0.0            0.0   
19996          1.057143  ...               28.0            1.0   
19997          2.000000  ...               12.0            0.0   
19998          1.000000  ...               28.0            1.0   
19999          1.000000  ...               28.0            1.0   

       voice_onnet_out_day_work_dur_m3  voice_onnet_in_day_work_dur_m3  \
0                              13860.0                         10498.0   
1                                360.0                          2860.0   
2                               2700.0                          1092.0   
3                               3420.0                          1714.0   
4                              16380.0                         15146.0   
...                                ...                             ...   
19995                              0.0                             0.0   
19996                          25200.0                         23608.0   
19997                              0.0                           720.0   
19998                          41220.0                         12369.0   
19999                          20400.0                          5784.0   

       voice_omo_out_night_rest_count_m3  sms_in_count_m3  content_count_m3  \
0                                    0.0             15.0              14.0   
1                                    0.0              0.0              28.0   
2                                    0.0              3.0              52.0   
3                                    0.0              6.0               5.0   
4                                    0.0             18.0              34.0   
.

In [35]:
scaler = StandardScaler()
df_new_50 = scaler.fit_transform(df_new_50)

In [36]:
df_new_50

array([[-0.49491972, -1.00193147, -0.89407506, ..., -0.15172515,
        -0.4140771 , -0.26601647],
       [-0.49491972, -1.5903347 ,  1.11847432, ..., -0.15172515,
        -0.37724397, -0.38109724],
       [-0.49491972,  1.03724168, -0.89407506, ..., -0.15172515,
        -0.45834618, -0.39559254],
       ...,
       [-0.49491972,  0.86588853,  1.11847432, ..., -0.15172515,
        -0.39035894, -0.38831039],
       [-0.49491972, -0.46331898, -0.89407506, ..., -0.15172515,
        -0.36318049, -0.39052684],
       [-0.49491972,  0.58538111,  1.11847432, ..., -0.15172515,
         1.00830965,  0.63451393]])

In [37]:
def count(df_y):
    a = df_y[df_y == 0].count()
    b = df_y[df_y == 1].count()
    print("Will not download", a)
    print("Will download", b)
    print("Not download / Will download", a/b)
    return a/b

In [38]:
def makeScore(model):
    roc_auc = roc_auc_score(y_test, model.predict(X_test))
    f1 = f1_score(y_test, model.predict(X_test))
    accuracy = accuracy_score(y_test, model.predict(X_test))
    recall = recall_score(y_test, model.predict(X_test))
    precision = precision_score(y_test, model.predict(X_test))

    print(confusion_matrix(y_test, model.predict(X_test)))
    print("ROC_AUC: %.5f%%" % (roc_auc * 100.0))
    print("F1_score: %.5f%%" % (f1 * 100.0))
    print("Accuracy: %.5f%%" % (accuracy * 100.0))
    print("Recall: %.5f%%" % (recall * 100.0))
    print("Precision: %.5f%% "% (precision * 100.0))

In [39]:
X_train, X_test, y_train, y_test = train_test_split(df_new_50, y, test_size=0.2, random_state=42)

In [41]:
print("\n\tIMBALANCED CLASS\n")
difference = count(y_train)
model = MLPClassifier(solver='adam', hidden_layer_sizes= 300, alpha=0.1, activation= 'logistic',random_state=42).fit(X_train, y_train)
print()
makeScore(model)


print("\n\tRANDOM OVERSAMPLING\n")
oversample = RandomOverSampler(random_state= 42, sampling_strategy = 'minority')
X_ros, y_ros = oversample.fit_resample(X_train, y_train)
model = MLPClassifier(solver='adam', hidden_layer_sizes= 300, alpha=0.1, activation= 'logistic',random_state=42).fit(X_ros, y_ros)
count(y_ros)
print()
makeScore(model)


print("\n\tRANDOM UNDERSAMPLING\n")
undersample = RandomUnderSampler(random_state= 42, sampling_strategy = 'majority')
X_rus, y_rus = undersample.fit_resample(X_train, y_train)
model = MLPClassifier(solver='adam', hidden_layer_sizes= 300, alpha=0.1, activation= 'logistic',random_state=42).fit(X_rus, y_rus)
count(y_rus)
print()
makeScore(model)


print("\n\tRANDOM OVERSEMPLING AND UNDERSAMPLING\n")
over = RandomOverSampler(random_state=42, sampling_strategy=0.2)
X_rous, y_rous = over.fit_resample(X_train, y_train)
under = RandomUnderSampler(random_state=42, sampling_strategy=0.9) # sampling_strategy = float - количество выборок в классе большинства после повторной выборки и количество выборок в классе меньшинства, соответственно.
X_rous, y_rous = under.fit_resample(X_rous, y_rous)
model = MLPClassifier(solver='adam', hidden_layer_sizes= 300, alpha=0.1, activation= 'logistic',random_state=42).fit(X_rous, y_rous)
count(y_rous)
print()
makeScore(model)


	IMBALANCED CLASS

Will not download 14792
Will download 1208
Not download / Will download 12.245033112582782

[[3704    8]
 [ 280    8]]
ROC_AUC: 51.28113%
F1_score: 5.26316%
Accuracy: 92.80000%
Recall: 2.77778%
Precision: 50.00000% 

	RANDOM OVERSAMPLING

Will not download 14792
Will download 14792
Not download / Will download 1.0

[[3033  679]
 [ 141  147]]
ROC_AUC: 66.37482%
F1_score: 26.39138%
Accuracy: 79.50000%
Recall: 51.04167%
Precision: 17.79661% 

	RANDOM UNDERSAMPLING

Will not download 1208
Will download 1208
Not download / Will download 1.0

[[2839  873]
 [ 120  168]]
ROC_AUC: 67.40751%
F1_score: 25.28217%
Accuracy: 75.17500%
Recall: 58.33333%
Precision: 16.13833% 

	RANDOM OVERSEMPLING AND UNDERSAMPLING

Will not download 3286
Will download 2958
Not download / Will download 1.1108857336037863

[[2737  975]
 [ 105  183]]
ROC_AUC: 68.63775%
F1_score: 25.31120%
Accuracy: 73.00000%
Recall: 63.54167%
Precision: 15.80311% 


In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [17]:
params={'eta': [0.01, 0.1, 0.2, 0.3], 'gamma':np.arange(0.1,0.6,0.1), "n_estimators": np.arange(50, 201, 50)}
model = XGBClassifier(scale_pos_weight = difference, use_rmm = 'true')

grid = GridSearchCV(model, params, cv=kf, scoring='f1', n_jobs = -1)
grid.fit(df_new_50, y)

[17:35:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weigh...
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None,
                                     scale_pos_wei

In [18]:
print("Найлучшее качество модели при", grid.best_params_)
print("Наилучшая оценка качества модели:", grid.best_score_)

Найлучшее качество модели при {'eta': 0.1, 'gamma': 0.5, 'n_estimators': 100}
Наилучшая оценка качества модели: 0.3169408771791812


In [23]:
steps = [('under', RandomUnderSampler(sampling_strategy = 'majority', random_state=42)), ('model', XGBClassifier(use_rmm = 'true'))]
pipeline = Pipeline(steps=steps)

params={
    'model__eta': [0.01, 0.1, 0.2, 0.3],
    'model__gamma':np.arange(0.1,0.6,0.1),
    'model__n_estimators': np.arange(50, 201, 50)
}
grid = GridSearchCV(pipeline, params, cv=kf, scoring='f1', n_jobs = -1)
grid.fit(df_new_50, y)

[17:50:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('under',
                                        RandomUnderSampler(random_state=42,
                                                           sampling_strategy='majority')),
                                       ('model',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      enable_categorical=False,
                                                      gamma=None, gpu_id=None,
                                                      importance_type=Non...
                                                      num_para

In [24]:
# scores = cross_val_score(pipeline, df_new_50, y, scoring='roc_auc', cv=kf, n_jobs=-1)
# print(scores)
# print(statistics.mean(scores))
print("Найлучшее качество модели при", grid.best_params_)
print("Наилучшая оценка качества модели:", grid.best_score_)

Найлучшее качество модели при {'model__eta': 0.1, 'model__gamma': 0.30000000000000004, 'model__n_estimators': 50}
Наилучшая оценка качества модели: 0.2765941466904811


In [25]:
steps = [('over', RandomOverSampler(sampling_strategy = 'minority', random_state=42)), ('model', XGBClassifier(use_rmm = 'true'))]
pipeline = Pipeline(steps=steps)

params={
    'model__eta': [0.01, 0.1, 0.2, 0.3],
    'model__gamma':np.arange(0.1,0.6,0.1),
    'model__n_estimators': np.arange(50, 201, 50)
}
grid = GridSearchCV(pipeline, params, cv=kf, scoring='f1', n_jobs = -1)
grid.fit(df_new_50, y)

[18:07:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('over',
                                        RandomOverSampler(random_state=42,
                                                          sampling_strategy='minority')),
                                       ('model',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      enable_categorical=False,
                                                      gamma=None, gpu_id=None,
                                                      importance_type=None,...
                                                      num_paral

In [26]:
print("Найлучшее качество модели при", grid.best_params_)
print("Наилучшая оценка качества модели:", grid.best_score_)

Найлучшее качество модели при {'model__eta': 0.1, 'model__gamma': 0.30000000000000004, 'model__n_estimators': 100}
Наилучшая оценка качества модели: 0.318228330265398


In [32]:
steps = [('over', RandomOverSampler(random_state=42, sampling_strategy=0.2)), ('under', RandomUnderSampler(random_state=42, sampling_strategy=0.9)) ,('model', XGBClassifier(use_rmm = 'true'))]
pipeline = Pipeline(steps=steps)

params={
    'model__eta': [0.01, 0.1, 0.2, 0.3],
    'model__gamma':np.arange(0.1,0.6,0.1),
    'model__n_estimators': np.arange(50, 201, 50)
}
grid = GridSearchCV(pipeline, params, cv=kf, scoring='f1', n_jobs = -1)
grid.fit(df_new_50, y)

[18:16:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('over',
                                        RandomOverSampler(random_state=42,
                                                          sampling_strategy=0.2)),
                                       ('under',
                                        RandomUnderSampler(random_state=42,
                                                           sampling_strategy=0.9)),
                                       ('model',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      enable_c...
                                 

In [33]:
print("Найлучшее качество модели при", grid.best_params_)
print("Наилучшая оценка качества модели:", grid.best_score_)

Найлучшее качество модели при {'model__eta': 0.1, 'model__gamma': 0.2, 'model__n_estimators': 200}
Наилучшая оценка качества модели: 0.32712765904731345
